In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import tkinter as tk
from tkinter import ttk
from tkinter import *
import re

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver1 = webdriver.Chrome(options=options)
driver1.get("https://registration.boun.edu.tr/buis/General/schedule.aspx?p=semester")

wait = WebDriverWait(driver1, 10)
element1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="submit"]')))
element_id1 = element1.get_attribute('id')
go_button = wait.until(EC.presence_of_element_located((By.ID, element_id1)))


go_button.click()

driver1.implicitly_wait(3)

link_elements = driver1.find_elements(By.TAG_NAME, 'a')

urls = []

for link in link_elements:
    urls.append(link.get_attribute('href'))

driver1.quit()


In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

def get_data():
    dfs = []

    for url in urls:
        driver.get(url)
        info_links = driver.find_elements(By.LINK_TEXT,"Info")
        for info_link in info_links:
            info_link.click()
            # Switch to the new window
            new_window = driver.window_handles[-1]
            driver.switch_to.window(new_window)
            try:
                # Find the table element
                table = driver.find_element(By.TAG_NAME, "table")
                # Convert the table content to a DataFrame
                df2 = pd.read_html(table.get_attribute('outerHTML'))[0]
                strongs = driver.find_elements(By.TAG_NAME, "strong")
                strongs_text = [strong.text for strong in strongs]
                strongs_text_repeated = np.repeat(strongs_text, len(df2))
                df2["Code"] = strongs_text_repeated[:len(df2)]
                dfs.append(df2)
            except IndexError:
                # No table found, move on to the next link
                pass
            finally:
                # Close the new window
                driver.close()
                # Switch back to the main window
                main_window = driver.window_handles[0]
                driver.switch_to.window(main_window)


    result = pd.concat(dfs, ignore_index=True)
    result = result[["Code",0,1,2,3,4]]
    result.drop(result.columns[[5]], axis=1, inplace=True)
    result.columns = ['Code', 'Department', 'Status', 'Quota', 'Current Quota']

    words = ["Departmental Quotas:", "Department", "OZELLISANS", "PHD","MASTER","OZELMASTER","OZELPHD"]

    result = result[result.Department.isin(words) == False]
    result = result[result.Status.isin(words) == False]
    result.reset_index(drop=True, inplace=True)

    def get_availability(row):
        quota = row['Quota']
        try:
            quota = int(quota)
        except ValueError:
            pass
        current_quota = int(row['Current Quota'])

            
        if isinstance(quota, int) and current_quota == quota:
            return "Full"
        elif isinstance(quota, int) and current_quota < quota:
            return "Available"
        elif isinstance(quota, int) and current_quota > quota:
            return "Try to get consent"
        elif isinstance(quota, str):
            return "Available"
        

    result["Availability"] = result.apply(get_availability, axis=1)
    return result




In [5]:
# table_data = get_data()
# table_data.to_csv("data2.csv", index=False)

In [16]:
testingdata = pd.read_csv("data1.csv")
# testingdata = get_data()

testingdata['Code'] = testingdata['Code'].str.replace(r'([A-Za-z]+)(\d)', r'\1 \2')

def update_table(code_filter='', availability_filter='', grade_filter='', search_filter="" ):
    
    filtered_data = testingdata.copy()
    if code_filter:
        filtered_data = filtered_data[filtered_data['Code'].str.contains(code_filter)]
    if availability_filter:
        filtered_data = filtered_data[filtered_data['Availability'] == availability_filter]
    if grade_filter:
        grade = int(grade_filter)
        filtered_data = filtered_data[filtered_data['Code'].str.contains(fr'\b{grade}\d{{2}}\b')]
    if search_filter:
        search_filter = search_filter.lower().replace(' ', '')
        filtered_data = filtered_data[filtered_data['Code'].str.lower().str.contains(search_filter)]
            
    table.delete(*table.get_children())  # clear the table
    for i, row in filtered_data.iterrows():
        table.insert('', 'end', values=row.tolist(), iid=i)


root = tk.Tk()
root.title("Course Availability")

table_frame = ttk.Frame(root)
table_frame.pack(padx=10, pady=10)

table = ttk.Treeview(table_frame, columns=['Code', 'Department', 'Status', 'Quota', 'Current Quota', 'Availability'])
table.heading('#0', text='Index')
table.heading('Code', text='Code')
table.heading('Department', text='Department')
table.heading('Status', text='Status')
table.heading('Quota', text='Quota')
table.heading('Current Quota', text='Current Quota')
table.heading('Availability', text='Availability')
table.column('#0', width=0, stretch=tk.NO)
table.column('Code', width=120, anchor=tk.CENTER)
table.column('Department', width=100, anchor=tk.CENTER)
table.column('Status', width=100, anchor=tk.CENTER)
table.column('Quota', width=100, anchor=tk.CENTER)
table.column('Current Quota', width=120, anchor=tk.CENTER)
table.column('Availability', width=120, anchor=tk.CENTER)
table_scroll = ttk.Scrollbar(table_frame, orient='vertical', command=table.yview)
table_scroll.pack(side='right', fill='y')
table.configure(yscrollcommand=table_scroll.set)
table.pack()


code_filter_var = tk.StringVar()
code_filter_var.set('')
code_filter_label = ttk.Label(root, text='Filter by Code:')
code_filter_label.pack()
code_filter_dropdown = ttk.Combobox(root, textvariable=code_filter_var, values=testingdata['Code'].str.extract(r'([A-Z]+)').dropna().drop_duplicates().values.squeeze().tolist() + [''], state='readonly')
code_filter_dropdown.pack()

availability_filter_var = tk.StringVar()
availability_filter_var.set('')
availability_filter_label = ttk.Label(root, text='Filter by Availability:')
availability_filter_label.pack()
availability_filter_dropdown = ttk.Combobox(root, textvariable=availability_filter_var, values=['Full', 'Available', 'Try to get consent', ''], state='readonly')
availability_filter_dropdown.pack()

grade_filter_var = tk.StringVar()
grade_filter_var.set('')
grade_filter_label = ttk.Label(root, text='Filter by Grade:')
grade_filter_label.pack()
grade_filter_dropdown = ttk.Combobox(root, textvariable=grade_filter_var, values=['1', '2', '3', '4', '5', '6', '7', ''], state='readonly')
grade_filter_dropdown.pack()

search_filter_var = tk.StringVar()
search_filter_var.set('')
search_filter_label = ttk.Label(root, text='Search:')
search_filter_label.pack()
search_filter_entry = ttk.Entry(root, textvariable=search_filter_var)
search_filter_entry.pack()

search_button = ttk.Button(root, text="Search", command=lambda: update_table(code_filter_var.get(), availability_filter_var.get(), grade_filter_var.get(), search_filter_var.get()))
search_button.pack(padx=10, pady=10)


# Insert data into the table
for i, row in testingdata.iterrows():
    table.insert('', 'end', values=row.tolist(), iid=i)

root.mainloop()

C:\Users\Aykut\AppData\Local\Temp\ipykernel_5108\2334021162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  testingdata['Code'] = testingdata['Code'].str.replace(r'([A-Za-z]+)(\d)', r'\1 \2')
